In [1]:
#-*- coding:utf-8 -*-
from commonTool import *


running on Windows


In [ ]:
#################################################
# EVALUATE-PREPARE.xml 실행되어야 함.

In [2]:
import glob


# 기준일 로딩

bf = open(resourceDir + 'BASIS-DAYS.txt', 'r', encoding='utf-8')

# 0.YYYYMM, 1.FIRST_DAY, 2.SECOND_DAY, 3.PREV_DAY, 4.LAST_DAY, 5.NEXT_DAY
lineText = bf.readline() # 제목

basisDays = []
quaterBasisDays = []

while True:
    lineText = bf.readline()
    if not lineText:
        break
        
    lineText = lineText.strip()
    if len(lineText) <= 0:
        continue

    dayData = lineText.split('\t')
    
    if dayData[0][0:4] == '2012' or dayData[0] == '201301': continue # 데이터 부족
        
    basisDays.append([dayData[0], dayData[3], dayData[4]])
    
    if dayData[0] > '202001':
        quaterBasisDays.append([dayData[0], dayData[3], dayData[4]])

bf.close()


In [3]:
# 로직 적용한 결과 정리

def makeAnalysisReport(resultFolder, initialCash, period, feeRate = 0.015, withProcess = True):
    statResult = [f for f in glob.glob(resultFolder + '*-STAT.txt')]
    analResult = [f for f in glob.glob(resultFolder + '*-RESULT.txt')]
    
    if len(statResult) == 0:
        print('no result file')
        return
    
    statResult.sort()
    
    m = 0
    y = 0
    cash = initialCash
    maxV = initialCash
    maxMdd = 0
    cagr = 0
    
    if withProcess:
        print('\t'.join(['Year', '매수일', '매도일', '손익', '투자금', '수익률', 'CAGR']))
    
    for statPath in statResult:
        m += period
        
        sf = open(statPath, 'r', encoding='utf-8')
        lineText = sf.readline() # 제목: --> 매수일, 매도일, R_SUM
        lineText = sf.readline() # 결과
        
        rec = lineText.split('\t')
        profitLoss = int(rec[2])

        newCash = round((cash + profitLoss) * (1.0 - feeRate))

        maxV = max(maxV, newCash)
        if newCash < cash:
            mdd = (newCash - maxV) / maxV
            maxMdd = min(mdd, maxMdd)

        y = m / 12
        cagr = pow(newCash / initialCash, 1 / y) - 1
        cagr = round(cagr * 100, 2)
        
        if withProcess:
            print('\t'.join([str(y), rec[0], rec[1],
                format(profitLoss, ',d').rjust(12), format(newCash, ',d').rjust(15),
                str(round(profitLoss / cash * 100, 2)).rjust(9), str(cagr).rjust(9)]))

        sf.close()
        cash = newCash

    if withProcess: print('')
    print('Year:', y, '  Invest:', format(initialCash, ',d'), '  Cash:', format(cash, ',d'),
        '  CAGR:', cagr, '%', '  Max MDD:', round(maxMdd * -100, 2), '%\n\n')


In [21]:
testResult = [
    '4PXX-25-10T-10-M2',
    '4PXX-25-10T-10-M2-P3',
    '4PXX-25-10T-15-M2',
    '4PXX-25-10T-20-M2',
    'END' # 콤마 편하게 하려고 넣었음
]

for key in testResult:
    if key == 'END': break
        
    print('#', key)
    makeAnalysisReport(outputRawPath + 'logicResult' + os.path.sep + key + os.path.sep, 10000000, 6, 0.015, True)


# 4PXX-25-10T-10-M2
Year	매수일	매도일	손익	투자금	수익률	CAGR
0.5	20130228	20130830	   1,804,682	     11,627,612	    18.05	     35.2
1.0	20130830	20140228	     943,980	     12,383,018	     8.12	    23.83
1.5	20140228	20140829	   3,306,281	     15,453,960	     26.7	    33.67
2.0	20140829	20150227	    -782,621	     14,451,269	    -5.06	    20.21
2.5	20150227	20150831	   1,184,610	     15,401,341	      8.2	    18.86
3.0	20150831	20160229	    -296,476	     14,878,292	    -1.93	    14.16
3.5	20160229	20160831	   4,609,739	     19,195,711	    30.98	    20.48
4.0	20160831	20170228	   8,982,607	     27,755,643	    46.79	    29.07
4.5	20170228	20170831	   4,574,559	     31,845,249	    16.48	    29.36
5.0	20170831	20180228	   7,914,065	     39,162,924	    24.85	    31.39
5.5	20180228	20180831	   9,515,329	     47,948,079	     24.3	    32.98
6.0	20180831	20190228	    -812,796	     46,428,254	     -1.7	    29.16
6.5	20190228	20190830	   7,018,282	     52,644,838	    15.12	    29.12
7.0	20190830	20200228	  -1,4

In [27]:
testResult = [
    # '4PXX-25-10T-20-M2',
    'Q-4PXX-25-10T-10-M2',
    'Q-4PXX-25-10T-10-M3',
    'Q-4PXX-25-10T-10-M4',
    'END' # 콤마 편하게 하려고 넣었음
]

for key in testResult:
    if key == 'END': break
        
    print('#', key)
    makeAnalysisReport(outputRawPath + 'logicResult' + os.path.sep + key + os.path.sep, 10000000, 3, 0.015, False)


# Q-4PXX-25-10T-10-M2
Year: 1.5   Invest: 10,000,000   Cash: 27,212,360   CAGR: 94.91 %   Max MDD: 0 %


# Q-4PXX-25-10T-10-M3
Year: 1.5   Invest: 10,000,000   Cash: 29,400,496   CAGR: 105.23 %   Max MDD: 1.5 %


# Q-4PXX-25-10T-10-M4
Year: 1.25   Invest: 10,000,000   Cash: 24,515,198   CAGR: 104.9 %   Max MDD: 0 %




In [22]:
# 년간 재무 데이터를 이용한 분석

analId = '4PXX'  # 로직명: 4PXX, MAGIC
inital_cash = 10000000  # 초기 투자금
period = 6  # 리밸런싱 기간 (월)
buyCount = 10  # 매수 종목 개수
totalAmount = 25 # 시가총액 하위 퍼센트 조건
volumeLimit = 10000 # 거래량 조건

startMonth = 2

analTitle = analId + '-' + str(totalAmount) + '-' + str(round(volumeLimit / 1000)) + 'T-' + str(buyCount) \
    + '-M' + str(startMonth) + ('-P' + str(period) if period != 6 else '')

print('running', analTitle)


feeRate = 0.015 # 1.5% 수수료

# 로직 스크립트
scriptPathName = crawlegoScriptPath + 'ANAL-' + analId + '.xml'

# 결과 저장 위치
resultFolder = outputRawPath + 'logicResult' + os.path.sep + analTitle + os.path.sep
# [os.remove(f) for f in glob.glob(resultFolder + '*.txt')]
mkdir(resultFolder)

# 계산 시작
m = 0
cash = inital_cash

for i in range(startMonth - 2, len(basisDays), period):
    basis = basisDays[i]
    sellIdx = min(i + period, len(basisDays) - 1)
    
    parameter = {
        'BASIS_DAY': basis[1],
        'BUY_DAY': basis[2],
        'SELL_DAY': basisDays[sellIdx][2],
        'CASH': str(cash),
        'TOP_N': str(buyCount),
        'PERIOD': 'ANNUAL',
        'IN_PATH': outputRawPath + 'index' + os.path.sep + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'TOTAL_AMOUNT': str(totalAmount),
        'VOL_LIMIT': str(volumeLimit),
        'DO_SERVER': '13.124.29.70'
    }

    m += period
    retCode = runDashScript(scriptPathName, parameter, False)
    
    print(m / 12, basis[0], 'ret code', retCode)
    
    if retCode != 0:
        continue

    # 결과 정리
    statPath = resultFolder + basis[1] + '-STAT.txt'
    sf = open(statPath, 'r', encoding='utf-8')
    
    lineText = sf.readline() # 제목: 매수일, 매도일, R_SUM
    lineText = sf.readline() # 결과
    
    rec = lineText.split('\t')
    profitLoss = int(rec[2])
    sf.close()
    
    cash = round((cash + profitLoss) * (1.0 - feeRate))
    # end of for

makeAnalysisReport(resultFolder, inital_cash, period, feeRate, True)
# end of script

running 4PXX-25-10T-10-M2-P3
0.25 201302 ret code 0
0.5 201305 ret code 0
0.75 201308 ret code 0
1.0 201311 ret code 0
1.25 201402 ret code 0
1.5 201405 ret code 0
1.75 201408 ret code 0
2.0 201411 ret code 0
2.25 201502 ret code 0
2.5 201505 ret code 0
2.75 201508 ret code 0
3.0 201511 ret code 0
3.25 201602 ret code 0
3.5 201605 ret code 0
3.75 201608 ret code 0
4.0 201611 ret code 0
4.25 201702 ret code 0
4.5 201705 ret code 0
4.75 201708 ret code 0
5.0 201711 ret code 0
5.25 201802 ret code 0
5.5 201805 ret code 0
5.75 201808 ret code 0
6.0 201811 ret code 0
6.25 201902 ret code 0
6.5 201905 ret code 0
6.75 201908 ret code 0
7.0 201911 ret code 0
7.25 202002 ret code 0
7.5 202005 ret code 0
7.75 202008 ret code 0
8.0 202011 ret code 0
8.25 202102 ret code 0
8.5 202105 ret code 0
Year	매수일	매도일	손익	투자금	수익률	CAGR
0.25	20130228	20130531	   2,117,502	     11,935,739	    21.18	   102.95
0.5	20130531	20130830	    -839,465	     10,929,830	    -7.03	    19.46
0.75	20130830	20131129	     -53,85

In [7]:
# 분기 재무 데이터를 이용한 분석

analId = '4PXX'  # 로직명
inital_cash = 10000000  # 초기 투자금
period = 3  # 리밸런싱 기간 (월)
buyCount = 10  # 매수 종목 개수
totalAmount = 25 # 시가총액 하위 퍼센트 조건
volumeLimit = 10000 # 거래량 조건

startMonth = 2

analTitle = 'Q-' + analId + '-' + str(totalAmount) + '-' + str(round(volumeLimit / 1000)) + 'T-' + str(buyCount) + '-M' + str(startMonth)

print('running', analTitle)


feeRate = 0.015 # 1.5% 수수료

# 로직 스크립트
scriptPathName = crawlegoScriptPath + 'ANAL-' + analId + '.xml'

# 결과 저장 위치
resultFolder = outputRawPath + 'logicResult' + os.path.sep + analTitle + os.path.sep
# [os.remove(f) for f in glob.glob(resultFolder + '*.txt')]
mkdir(resultFolder)

# 계산 시작
m = 0
cash = inital_cash

for i in range(startMonth - 2, len(quaterBasisDays), period):
    basis = quaterBasisDays[i]
    sellIdx = min(i + period, len(quaterBasisDays) - 1)
    
    parameter = {
        'BASIS_DAY': basis[1],
        'BUY_DAY': basis[2],
        'SELL_DAY': quaterBasisDays[sellIdx][2],
        'CASH': str(cash),
        'TOP_N': str(buyCount),
        'PERIOD': 'QUARTER',
        'IN_PATH': outputRawPath + 'index' + os.path.sep + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'TOTAL_AMOUNT': str(totalAmount),
        'VOL_LIMIT': str(volumeLimit),
        'DO_SERVER': '13.124.29.70'
    }

    m += period
    retCode = runDashScript(scriptPathName, parameter, False)
    
    print(m / 12, basis[0], 'ret code', retCode)
    
    if retCode != 0:
        continue

    # 결과 정리
    statPath = resultFolder + basis[1] + '-STAT.txt'
    sf = open(statPath, 'r', encoding='utf-8')
    
    lineText = sf.readline() # 제목: 매수일, 매도일, R_SUM
    lineText = sf.readline() # 결과
    
    rec = lineText.split('\t')
    profitLoss = int(rec[2])
    sf.close()
    
    cash = round((cash + profitLoss) * (1.0 - feeRate))
    # end of for

makeAnalysisReport(resultFolder, inital_cash, period, feeRate, True)
# end of script

running Q-4PXX-25-10T-10-M2
0.25 202002 ret code 0
0.5 202005 ret code 0
0.75 202008 ret code 0
1.0 202011 ret code 0
1.25 202102 ret code 0
1.5 202105 ret code 0
Year	매수일	매도일	손익	투자금	수익률	CAGR
0.25	20200228	20200529	   1,455,550	     11,283,717	    14.56	    62.11
0.5	20200529	20200831	   3,357,570	     14,421,668	    29.76	   107.98
0.75	20200831	20201130	   2,278,000	     16,449,173	     15.8	    94.17
1.0	20201130	20210226	   2,718,600	     18,880,256	    16.53	     88.8
1.25	20210226	20210531	   7,563,720	     26,047,316	    40.06	   115.09
1.5	20210531	20210621	   1,579,445	     27,212,360	     6.06	    94.91

Year: 1.5   Invest: 10,000,000   Cash: 27,212,360   CAGR: 94.91 %   Max MDD: 0 %


